# RAG (main)

## Dependencies

In [198]:
import os
from dotenv import load_dotenv
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

In [199]:
# load the environment variables
load_dotenv() 

True

In [200]:
# setup

# LangChain
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_ENDPOINT"] = os.getenv("LANGCHAIN_ENDPOINT")
os.environ["LANGCHAIN_PROJECT"] = os.getenv("LANGCHAIN_PROJECT")

# OpenAI
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

## Documents

Things to Consider
- chunking level
- data cleaning
- meta data

In [201]:
from langchain_community.document_loaders import PyPDFLoader

pdf_paths = [
    "https://www.ph.emb-japan.go.jp/files/100508281.pdf", # TOURISM
    "https://www.ph.emb-japan.go.jp/files/100412012.pdf", # PACKAGE TOUR
    "https://www.ph.emb-japan.go.jp/files/100508282.pdf", # BUSINESS, CONFERENCE or CULTURAL EXCHANGE, etc.
    "https://www.ph.emb-japan.go.jp/files/100508283.pdf", # VISITING RELATIVES
    "https://www.ph.emb-japan.go.jp/files/100508284.pdf", # VISITING FRIENDS OR DISTANT RELATIVES
    "https://www.ph.emb-japan.go.jp/files/100508285.pdf", # VISITING US MILITARY PERSONNEL
    "https://www.ph.emb-japan.go.jp/files/100585068.pdf", # SPOUSE OR CHLID OF JAPANESE NATIONAL RESIDING IN THE PHILIPPINES
    "https://www.ph.emb-japan.go.jp/files/100508287.pdf", # TRANSIT
    "https://www.ph.emb-japan.go.jp/files/100508288.pdf", # MULTIPLE-ENTRY VISA FOR TEMPORARY VISITOR
    "https://www.ph.emb-japan.go.jp/files/100674192.pdf", # MULTIPLE-ENTRY TEMPORARY VISITOR VISA (PHILIPPINE NATIONALS WITH CONSIDERABLE FINANCIAL CAPACITY) 
    "https://www.ph.emb-japan.go.jp/files/100404404.pdf", # MULTIPLE-ENTRY VISA FOR TEMPORARY VISITOR
    "https://www.ph.emb-japan.go.jp/files/100479463.pdf", # STUDENT, WORKER AND DEPENDENT
    "https://www.ph.emb-japan.go.jp/files/100415046.pdf", # OFFICIAL 
    "https://www.ph.emb-japan.go.jp/files/100415047.pdf", # HOUSEKEEPER OF DIPLOMAT/OFFICIAL
    "https://www.ph.emb-japan.go.jp/files/100415048.pdf", # NIKKEI-JIN (JAPANESE DESCENDANT)
    "https://www.ph.emb-japan.go.jp/files/100508289.pdf", # FILIPINO PARENTS TRAVELLING TO JAPAN WITH JAPANESE-FILIPINO CHILDREN
]
headers = {
    "User-Agent": "Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/131.0.0.0 Mobile Safari/537.36"
    }

pages = []
for pdf_path in pdf_paths:
    loader = PyPDFLoader(pdf_path, headers=headers)
    async for page in loader.alazy_load():
        pages.append(page)

In [202]:
pages[0]

Document(metadata={'source': 'https://www.ph.emb-japan.go.jp/files/100508281.pdf', 'page': 0}, page_content='TOURISM   \nMay 2024  \n  \nA. PURPOSE  \nVisit Japan for tourism.  \n  \nB. Requirements（Details→https://www.ph.emb-japan.go.jp/itpr_ja/11_000001_00898.html）  \n    \n※ Downloadable from this website   \n      \n(1) Passport（Holder’s signature required）  \n(2) Application Form ※（A facial Photo (4.5×3.5cm) must be attached.）  \n(3) PSA issued Birth Certificate and Marriage Certificate (for married applicants), issued within 1 year \n☞ Unnecessary if there is used Japan Visa on passport. \n【ADDITIONAL REQUIREMENTS】  \n- If (3) is unreadable, submit Birth/Marriage certificate issued by Local Civil Registrar.  \n- If Birth Certificate is “LATE REGISTRATION”, submit Baptismal Certificate and School Record (Form 137).  \n-If there is no record of Birth/Marriage in PSA, submit Birth Certificate issued by Local Civil Registrar and Negative  \nCertificate issued by PSA.  \n(4) Itinerary

In [203]:
print(pages[0])

page_content='TOURISM   
May 2024  
  
A. PURPOSE  
Visit Japan for tourism.  
  
B. Requirements（Details→https://www.ph.emb-japan.go.jp/itpr_ja/11_000001_00898.html）  
    
※ Downloadable from this website   
      
(1) Passport（Holder’s signature required）  
(2) Application Form ※（A facial Photo (4.5×3.5cm) must be attached.）  
(3) PSA issued Birth Certificate and Marriage Certificate (for married applicants), issued within 1 year 
☞ Unnecessary if there is used Japan Visa on passport. 
【ADDITIONAL REQUIREMENTS】  
- If (3) is unreadable, submit Birth/Marriage certificate issued by Local Civil Registrar.  
- If Birth Certificate is “LATE REGISTRATION”, submit Baptismal Certificate and School Record (Form 137).  
-If there is no record of Birth/Marriage in PSA, submit Birth Certificate issued by Local Civil Registrar and Negative  
Certificate issued by PSA.  
(4) Itinerary in Japan  
 
【In case that applicant will shoulder part/all of travel expense】   
 
(5) Applicant’s Bank Certific

## Embeddings

Things to Consider
- Evaluation Metric
- dataset
- model

In [204]:
embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

In [205]:
embeddings

OpenAIEmbeddings(client=<openai.resources.embeddings.Embeddings object at 0x1579fbbd0>, async_client=<openai.resources.embeddings.AsyncEmbeddings object at 0x157d3e490>, model='text-embedding-3-large', dimensions=None, deployment='text-embedding-ada-002', openai_api_version=None, openai_api_base=None, openai_api_type=None, openai_proxy=None, embedding_ctx_length=8191, openai_api_key=SecretStr('**********'), openai_organization=None, allowed_special=None, disallowed_special=None, chunk_size=1000, max_retries=2, request_timeout=None, headers=None, tiktoken_enabled=True, tiktoken_model_name=None, show_progress_bar=False, model_kwargs={}, skip_empty=False, default_headers=None, default_query=None, retry_min_seconds=4, retry_max_seconds=20, http_client=None, http_async_client=None, check_embedding_ctx_length=True)

## Custom embeddings

In [206]:
from langchain_core.embeddings import Embeddings
import torch
from transformers import RobertaTokenizer, RobertaModel

# Load pre-trained RoBERTa model
model = RobertaModel.from_pretrained('roberta-base')

# Load the model and tokenizer from the saved directory
custom_model = RobertaModel.from_pretrained('./fine_tuned_roberta')
tokenizer = RobertaTokenizer.from_pretrained('./fine_tuned_roberta')


class RoBERTaEmbedding(Embeddings):
    def __init__(self, model_name='roberta-base'):
        # Load pre-trained RoBERTa model and tokenizer
        self.model = model
        self.tokenizer = tokenizer
    
    def embed_documents(self, documents):
        embeddings = []
        for doc in documents:
            inputs = self.tokenizer(doc, return_tensors='pt', truncation=True, padding=True, max_length=512)
            with torch.no_grad():
                outputs = self.model(**inputs)
                # Using the mean of the last hidden state as the document embedding
                embedding = outputs.last_hidden_state.mean(dim=1).squeeze().numpy()
                embeddings.append(embedding)
        return embeddings

    def embed_query(self, query):
        inputs = self.tokenizer(query, return_tensors='pt', truncation=True, padding=True, max_length=512)
        with torch.no_grad():
            outputs = self.model(**inputs)
            # Using the mean of the last hidden state as the query embedding
            embedding = outputs.last_hidden_state.mean(dim=1).squeeze().numpy()
        return embedding

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at ./fine_tuned_roberta and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [207]:
# Initialize the custom embedding class with your fine-tuned model and tokenizer
custom_embedding = RoBERTaEmbedding()

## Vector Database

Things to Consider
- Evaluation Metric
- HNSW parameters

In [220]:
from langchain_core.vectorstores import InMemoryVectorStore

vector_store = InMemoryVectorStore.from_documents(pages, embeddings)

In [209]:
# check
docs = vector_store.similarity_search("What is the requirements for tourism?", k=7)
for doc in docs:
    print(f'Page {doc.metadata["page"]}: {doc.page_content}\n')

Page 2: (4) We might request  to submit additional docume nts for examinations . If you don’t submit them within 3 
months from the date of request , we will cease the examination and return your passport.  
(5) We might  have an interview with applicant or his/her relatives. In such a case, we wil l take a contact 
with appli cant directly or through accredited agencies/ registered Nikkei-Jin agencies to arrange  the date 
and time of interview.    

Page 1: B. Requirements（Details→https://www.ph.emb-japan.go.jp/itpr_ja/11_000001_00898.html）  
submitted. For students, a copy of School ID or Certificate of Enrollment in School must be 
submitted.  
  
II. Spouse or Child of (I) 
(1) Proof of relationship with (I) above (PSA issued Birth Certificate and Marriage Certificate, if 
applicable)  
(2) Passport and multiple entry visa copy or documents of (I) above.  

Page 1: *Used Japanese Visa and landing permission as temporary visitor in the last 3 years    
*Used visas and landing permi

## Prompt Template

In [210]:
system_template = """
Answer the following based on this {context}, 
otherwise just give this url (https://www.ph.emb-japan.go.jp/itpr_en/00_000035.html) 
for more information about Japan Visa"""

prompt_template = ChatPromptTemplate.from_messages(
    [("system", system_template), ("user", "{question}")]
)

## LLM

In [211]:
llm = ChatOpenAI(model="gpt-4o-mini")

## RAG

Things to Consider
- Evaluation Metric

## Evaluation (BLUE SCORE)

In [221]:
# Define a function to change the question and generate the response
def change_question_and_generate_response(question):
    # Dynamically change the question
    
    # RAG model
    context = vector_store.similarity_search(question, k=1)
    # Invoke the prompt template with the new question
    prompt = prompt_template.invoke({
        "context": context,
        "question": question  # Provide the new question here
    })
    
    # Generate response from the LLM
    response = llm.invoke(prompt)
    
    return response

In [222]:
import spacy

# Load spaCy model
nlp = spacy.load("en_core_web_sm")

# Function to tokenize words using spaCy
def tokenize_text(text):
    # Process the text with spaCy NLP pipeline
    doc = nlp(text)
    # Extract words (tokens)
    tokens = [token.text for token in doc]
    return tokens

In [223]:
import pandas as pd


QA_PATH = "./dataset/qa/QA.xlsx"
qa = pd.read_excel(QA_PATH)
qa

,question,answer 1,answer 2,answer 3
0,What is the requirements for tourism?,"The requirements for tourism to Japan, as outl...",Here’s a simplified version of the requirement...,Here’s an even simpler version of the requirem...
1,What is the requirements for visiting relatives?,The requirements for visiting relatives in Jap...,Here’s a simplified version of the requirement...,Here’s an even simpler version of the requirem...
2,what is the requirements for business puposes?,The requirements for visiting Japan for **busi...,Here’s a simplified version of the requirement...,Here’s an even simpler version:\n\n### **For t...
3,what is the requirements for visiting friends?,The requirements for visiting Japan for **frie...,Here’s a simpler version:\n\n### **For the App...,Here’s an even simpler version:\n\n### **For t...
4,what is the requirements for visiting US milit...,The requirements for visiting **US military pe...,Here’s a simplified version:\n\n### **For the ...,Here’s the simplest version:\n\n### **For the ...
5,what is the requirements for spouse or child o...,The requirements for the **spouse or child of ...,Here’s a simpler version:\n\n### **For the App...,Here’s a more simplified version:\n\n### **For...
6,what is the requirements for transit?,The requirements for **transit** to Japan are ...,Here’s a simplified version for **transit** to...,Here’s the simplest version for **transit** to...


In [224]:
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction

bleu_scores = []
# Using SmoothingFunction() to prevent zero BLEU score
smooth = SmoothingFunction().method1
for row in qa.itertuples():
    question, answer1, answer2, answer3 = row[1], row[2], row[3], row[4]
    references = [tokenize_text(answer1), tokenize_text(answer2), tokenize_text(answer3)]
    # do something
    # Calculate BLEU score (use sentence_bleu for a single sentence)
    response = change_question_and_generate_response(question)
    candidate = tokenize_text(response.content)

    bleu_score = sentence_bleu(references, candidate, smoothing_function=smooth)
    bleu_scores.append(bleu_score)

print(f"average bleu score: {sum(bleu_scores)/len(bleu_scores)}")
    

average bleu score: 0.5229361315213505
